In [1]:
import pickle
import numpy as np

In [2]:
with open('train_qa.txt', 'rb') as f:
    train_data = pickle.load(f)

In [3]:
with open('test_qa.txt', 'rb') as f:
    test_data = pickle.load(f)

In [4]:
type(test_data)

list

In [5]:
type(train_data)

list

In [6]:
len(test_data)

1000

In [7]:
len(train_data)

10000

In [8]:
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [9]:
' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [10]:
' '.join(train_data[0][2])

'n o'

In [11]:
all_data = test_data + train_data

In [12]:
len(all_data)

11000

In [13]:
vocab = set()
for story,question,answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))
    vocab = vocab.union(set(answer))

In [14]:
vocab.add('no')

In [15]:
vocab.add('yes')

In [16]:
vocab

{'.',
 '?',
 'Daniel',
 'Is',
 'John',
 'Mary',
 'Sandra',
 'apple',
 'back',
 'bathroom',
 'bedroom',
 'discarded',
 'down',
 'dropped',
 'e',
 'football',
 'garden',
 'got',
 'grabbed',
 'hallway',
 'in',
 'journeyed',
 'kitchen',
 'left',
 'milk',
 'moved',
 'n',
 'no',
 'o',
 'office',
 'picked',
 'put',
 's',
 'the',
 'there',
 'to',
 'took',
 'travelled',
 'up',
 'went',
 'y',
 'yes'}

In [17]:
vocab_len = len(vocab)+1

In [18]:
vocab_len

43

In [19]:
#LONGEST STORY
all_story_lens = [len(data[0]) for data in all_data]

In [20]:
max_story_len = max(all_story_lens)

In [21]:
max_question_len = max([len(data[1]) for data in all_data])

In [22]:
max_question_len 

6

In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [28]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [30]:
tokenizer.word_index

{'down': 1,
 'no': 2,
 's': 3,
 'n': 4,
 'left': 5,
 'moved': 6,
 'journeyed': 7,
 'daniel': 8,
 'john': 9,
 'yes': 10,
 'got': 11,
 'grabbed': 12,
 'o': 13,
 'took': 14,
 'office': 15,
 'sandra': 16,
 'discarded': 17,
 'kitchen': 18,
 'there': 19,
 'dropped': 20,
 'went': 21,
 'bedroom': 22,
 'hallway': 23,
 'the': 24,
 'apple': 25,
 'up': 26,
 'bathroom': 27,
 'to': 28,
 'garden': 29,
 'mary': 30,
 'back': 31,
 'put': 32,
 '.': 33,
 'picked': 34,
 '?': 35,
 'e': 36,
 'y': 37,
 'travelled': 38,
 'football': 39,
 'is': 40,
 'in': 41,
 'milk': 42}

In [34]:
train_story_text = []
train_question_text = []
train_answers = [] 

In [40]:
for story,question,answer in train_data :
    train_story_text.append(story)
    train_question_text.append(question)
    train_answers.append(answer)

In [44]:
# train_story_text

In [50]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [52]:
len(train_story_seq)

10002

In [64]:
def vectorize_stories(data,word_index=tokenizer.word_index,max_story_len=max_story_len,max_question_len=max_question_len):
    # Stories = X
    X =[]
    # Questions Xq
    Xq = []
    # Y Correct Answer (yes/no)
    Y = []
    for story,query,answer in data:
        
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in query]

        y = np.zeros(len(word_index)+1)
        y[word_index[answer]] = 1
        
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X,maxlen=max_story_len),pad_sequences(Xq,maxlen=max_question_len),np.array(Y))     

In [66]:
inputs_train , queries_trian , answers_train = vectorize_stories(train_data)

In [70]:
inputs_test,queries_test,answers_test = vectorize_stories(test_data)

In [72]:
inputs_test

array([[ 0,  0,  0, ..., 24, 22, 33],
       [ 0,  0,  0, ..., 24, 29, 33],
       [ 0,  0,  0, ..., 24, 29, 33],
       ...,
       [ 0,  0,  0, ..., 24, 25, 33],
       [ 0,  0,  0, ..., 24, 29, 33],
       [ 0,  0,  0, ..., 25, 19, 33]])

In [74]:
answers_test

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [76]:
tokenizer.word_index['yes']

10

In [78]:
tokenizer.word_index['no']

2

In [82]:
sum(answers_test)

array([  0.,   0., 503.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 497.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [84]:
from tensorflow.keras.models import Sequential,Model

In [96]:
from tensorflow.keras.layers import Embedding

In [98]:
from tensorflow.keras.layers import Input,Activation,Dense,Permute,Dropout,add,dot,concatenate,LSTM

In [102]:
# PLACEHOLDER shape=(max_story_len,batch_size)
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [104]:
# vocab_len
vocab_size = len(vocab) + 1

In [106]:
# Input ENCODER M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))
# OUTPUT
#(samples,story_maxlen,embedding_dim)

In [108]:
# Input ENCODER C
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))
# OUTPUT
#(samples,story_maxlen,max_question_len)

In [110]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))
#(samples,query_maxlen,embedding_dim)

C:\Users\Engel\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [128]:
#ENCODED <----- ENCODER(INPUT)
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [130]:
match =dot([input_encoded_m,question_encoded],axes=(2,2))
match = Activation('softmax')(match)

In [132]:
response = add([match,input_encoded_c])
response = Permute((2,1))(response)

In [134]:
answer = concatenate([response,question_encoded])

In [136]:
answer

<KerasTensor shape=(None, 6, 220), dtype=float32, sparse=False, ragged=False, name=keras_tensor_29>

In [138]:
answer = Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer) #(samples,vocab_size) # yes/no 0000

In [140]:
answer = Activation('softmax')(answer)

In [142]:
model = Model([input_sequence,question],answer)

In [144]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

In [146]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 156)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_3 (InputLayer)    │ (None, 6)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential (Sequential)       │ (None, 156, 64)           │           2,752 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_2 (Sequential)     │ (None, 6, 64)             │           2,752 │ input_layer_3[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dot_1 (Dot)                   │ (None, 156, 6)            │               0 │ sequential[1][0],          │
│                               │                           │                 │ sequential_2[1][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_1 (Activation)     │ (None, 156, 6)            │               0 │ dot_1[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ sequential_1 (Sequential)     │ (None, 156, 6)            │             258 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_3 (Add)                   │ (None, 156, 6)            │               0 │ activation_1[0][0],        │
│                               │                           │                 │ sequential_1[1][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ permute_2 (Permute)           │ (None, 6, 156)            │               0 │ add_3[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 6, 220)            │               0 │ permute_2[0][0],           │
│                               │                           │                 │ sequential_2[1][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_3 (Dropout)           │ (None, 6, 220)            │               0 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 6, 43)             │           9,503 │ dropout_3[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_2 (Activation)     │ (None, 6, 43)             │               0 │ dense[0][0]                │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 15,265 (59.63 KB)

 Trainable params: 15,265 (59.63 KB)

 Non-trainable params: 0 (0.00 B)